In [1]:
import re
from bs4 import BeautifulSoup

### Split warc file to docs

In [2]:
# ref : https://stackoverflow.com/questions/60269904/split-text-file-after-specific-line-in-python
SECTION_START = re.compile(r'<!DOCTYPE html')
SECTION_END = re.compile(r'</html>')

def split_docs_iter(stream):
    def inner(stream):
        # Yields each line until an end marker is found (or EOF)
        for line in stream:
            if line and not SECTION_END.match(line):
                yield line
                continue
            break

    # Find a start marker, then break off into a nested iterator
    for line in stream:
        if line:
            if SECTION_START.match(line):
                yield inner(stream)
            continue
        break

In [3]:
filename = "03.warc"

# split docs
with open(filename, 'r', encoding="ISO-8859-1") as fh_in:
    for (i, nested_iter) in enumerate(split_docs_iter(fh_in)):
        with open('./docs/docID_{:05d}'.format(i), 'w', encoding='UTF-8') as fh_out:
            for line in nested_iter:
                fh_out.write(line)

### Parse html and get text in <body> tag 

In [6]:
# get text in <body>
from os import listdir
from os.path import isfile, join

PATH = 'docs/'

files = [f for f in listdir(PATH) if isfile(join(PATH, f))]

# with open("docs/docID_13180", 'r', encoding="ISO-8859-1") as f:
for file in files:
    with open(file, 'r', encoding="ISO-8859-1") as f:
        soup = BeautifulSoup(f, "html.parser")

        b = soup.find('body').getText()
        s = " ".join(b.split())
        w = re.sub(r"[^A-Za-z0-9]+", ' ', s)

        with open(f"{file}_", "w", "utf-8") as f1:
            file.write(w)





Home





The Daisy
A scrap-blog about a glance of my life



Feed on
Posts
Comments








Flying phobia
March 10, 2007 by Daisy Natalia Awondatu 


A new kind of illness is found. It called flying phobia. Based on daistionary (daisy dictionary), flying phobia is an unusual fear of flying. Well, this kind off illness appears after all of the accident happened to our commercial airlines company. I am really afraid to fly now. I think it is not only me that being afraid of flying, but also most of Indonesian people. Since the missing of Adam Air on January, I and my friends are afraid to fly. One of my friends prefer to have a long trip by car than flying, when she wants to go back to Bandung, although it takes almost 16 hours to go there from Situbondo. She is really afraid to fly. She thinks it is far better for her to have a hot seat, 16 hours of bended legs, than takes an only 1 our of flying to Bandung.Â 

The story is not yet end. The missing of Adam Air is followed with the l

In [29]:
# print(b.strip())
s = " ".join(b.split())
print(s)

Home The Daisy A scrap-blog about a glance of my life Feed on Posts Comments Flying phobia March 10, 2007 by Daisy Natalia Awondatu A new kind of illness is found. It called flying phobia. Based on daistionary (daisy dictionary), flying phobia is an unusual fear of flying. Well, this kind off illness appears after all of the accident happened to our commercial airlines company. I am really afraid to fly now. I think it is not only me that being afraid of flying, but also most of Indonesian people. Since the missing of Adam Air on January, I and my friends are afraid to fly. One of my friends prefer to have a long trip by car than flying, when she wants to go back to Bandung, although it takes almost 16 hours to go there from Situbondo. She is really afraid to fly. She thinks it is far better for her to have a hot seat, 16 hours of bended legs, than takes an only 1 our of flying to Bandung.Â The story is not yet end. The missing of Adam Air is followed with the landing problem of Lion A

In [30]:
w = re.sub(r"[^A-Za-z0-9]+", ' ', s)
print(w)

Home The Daisy A scrap blog about a glance of my life Feed on Posts Comments Flying phobia March 10 2007 by Daisy Natalia Awondatu A new kind of illness is found It called flying phobia Based on daistionary daisy dictionary flying phobia is an unusual fear of flying Well this kind off illness appears after all of the accident happened to our commercial airlines company I am really afraid to fly now I think it is not only me that being afraid of flying but also most of Indonesian people Since the missing of Adam Air on January I and my friends are afraid to fly One of my friends prefer to have a long trip by car than flying when she wants to go back to Bandung although it takes almost 16 hours to go there from Situbondo She is really afraid to fly She thinks it is far better for her to have a hot seat 16 hours of bended legs than takes an only 1 our of flying to Bandung The story is not yet end The missing of Adam Air is followed with the landing problem of Lion Air and the cracking of 